In [ ]:
import collections
import os
import json
from difflib import SequenceMatcher as SM
from itertools import combinations
from datetime import date
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
import pandas as pd
from collections import OrderedDict
from gspread_formatting import *
import string

In [ ]:
fname_grant_jsons = os.listdir("/Users/luisa/pub_tracker_backup/collate_revisited_test/data_grant")
for fname_json in fname_grant_jsons:
    with open("/Users/luisa/pub_tracker_backup/collate_revisited_test/data_grant/" + fname_json, "r") as fp:
        dic_pergrant = json.load(fp)
        pubs = dic_pergrant.keys()
        grant_name = fname_json[:-5]
        pubs_when_grant_primary[grant_name] = collections.defaultdict(dict)
        pubs_when_grant_secondary[grant_name] = collections.defaultdict(dict)
        for pub in pubs:
            authors = "; ".join(dic_pergrant[pub]["authors"])
            title = dic_pergrant[pub]["title"]
            link = dic_pergrant[pub]["link"]
            date = dic_pergrant[pub]["date"]
            other_ids = ";".join(dic_pergrant[pub]["other_ids"])
            primary_awards = ";".join(dic_pergrant[pub]["primary_awards"])
            secondary_awards = ";".join(dic_pergrant[pub]["secondary_awards"])
            contact_4DN_author = dic_pergrant[pub]["4DN_member_author"]
            added_date = dic_pergrant[pub]["added"]

            pubs_when_grant_primary[grant_name][pub]['title'] = title
            pubs_when_grant_primary[grant_name][pub]['authors'] = authors
            pubs_when_grant_primary[grant_name][pub]['date'] = date
            pubs_when_grant_primary[grant_name][pub]['link'] = link
            pubs_when_grant_primary[grant_name][pub]['other_ids'] = other_ids
            pubs_when_grant_primary[grant_name][pub]['primary_awards'] = primary_awards
            pubs_when_grant_primary[grant_name][pub]['secondary_awards'] = secondary_awards
            pubs_when_grant_primary[grant_name][pub]['date_added'] = added_date
            pubs_when_grant_primary[grant_name][pub]['contact_4DN_author'] = contact_4DN_author
            
#     # Keeps track of the grants of the publications secondary awards
    
    for grant in pubs_when_grant_primary:
        for pub in pubs_when_grant_primary[grant]:
            if pubs_when_grant_primary[grant][pub]['secondary_awards'] != '':
                if ';' not in pubs_when_grant_primary[grant][pub]['secondary_awards']:
                    sec_award = pubs_when_grant_primary[grant][pub]['secondary_awards']
                    pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub]
                else:
                    sec_award_list = pubs_when_grant_primary[grant][pub]['secondary_awards'].split(';')
                    for sec_award in sec_award_list:
                        pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub]

In [ ]:
working_forder = '/Users/luisa/pub_tracker_backup/data_grant' # The data_grant folder

In [ ]:
pubs_when_grant_primary = collections.defaultdict(dict)
pubs_when_grant_secondary_tracker = collections.defaultdict(dict)
pubs_when_grant_secondary = collections.defaultdict(dict)

fname_grant_jsons = os.listdir("/Users/luisa/pub_tracker_backup/collate_revisited_test/data_grant")
for fname_json in fname_grant_jsons:
    with open("/Users/luisa/pub_tracker_backup/collate_revisited_test/data_grant/" + fname_json, "r") as fp:
        dic_pergrant = json.load(fp)
        pubs = dic_pergrant.keys()
        grant_name = fname_json[:-5]
        pubs_when_grant_primary[grant_name] = collections.defaultdict(dict)
        pubs_when_grant_secondary[grant_name] = collections.defaultdict(dict)
        for pub in pubs:
            authors = "; ".join(dic_pergrant[pub]["authors"])
            title = dic_pergrant[pub]["title"]
            link = dic_pergrant[pub]["link"]
            date = dic_pergrant[pub]["date"]
            other_ids = ";".join(dic_pergrant[pub]["other_ids"])
            primary_awards = ";".join(dic_pergrant[pub]["primary_awards"])
            secondary_awards = ";".join(dic_pergrant[pub]["secondary_awards"])
            contact_4DN_author = dic_pergrant[pub]["4DN_member_author"]
            added_date = dic_pergrant[pub]["added"]

            pubs_when_grant_primary[grant_name][pub]['title'] = title
            pubs_when_grant_primary[grant_name][pub]['authors'] = authors
            pubs_when_grant_primary[grant_name][pub]['date'] = date
            pubs_when_grant_primary[grant_name][pub]['link'] = link
            pubs_when_grant_primary[grant_name][pub]['other_ids'] = other_ids
            pubs_when_grant_primary[grant_name][pub]['primary_awards'] = primary_awards
            pubs_when_grant_primary[grant_name][pub]['secondary_awards'] = secondary_awards
            pubs_when_grant_primary[grant_name][pub]['date_added'] = added_date
            pubs_when_grant_primary[grant_name][pub]['contact_4DN_author'] = contact_4DN_author
            
#     # Keeps track of the grants of the publications secondary awards
    
    for grant in pubs_when_grant_primary:
        for pub in pubs_when_grant_primary[grant]:
            if pubs_when_grant_primary[grant][pub]['secondary_awards'] != '':
                if ';' not in pubs_when_grant_primary[grant][pub]['secondary_awards']:
                    sec_award = pubs_when_grant_primary[grant][pub]['secondary_awards']
                    pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub]
                else:
                    sec_award_list = pubs_when_grant_primary[grant][pub]['secondary_awards'].split(';')
                    for sec_award in sec_award_list:
                        pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub]

In [ ]:
# Accesses to the google sheets
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)
book_id = '1PqqMQn7R0qbYm0bUOf3LwCtPJryIbo7YoRf2_PZxOQY' # This is the test spreadsheet
test_spreadsheet = gc.open_by_key(book_id)

In [ ]:
worksheet = test_spreadsheet.worksheet("Publication_List_new_format_updated")

In [ ]:
worksheet_values = worksheet.get_all_values()

In [ ]:
import pandas as pd
df = pd.DataFrame(worksheet_values[1:], columns=worksheet_values[0])
col_names = df.columns.values

In [ ]:
from collections import OrderedDict
google_sheet = df.to_dict(orient='records', into=OrderedDict)

In [ ]:
x = 0
for item in google_sheet:
    print(item)
    x = x + 1
    if x == 1:
        break

In [ ]:
test_grant_primary = pubs_when_grant_primary['DK107980'] # Test a specific grant

In [ ]:
print(test_grant_primary)

In [ ]:
print(len(test_grant_primary))

In [ ]:
# Comparing primary awards with existing google sheets and adding new awards

In [ ]:
new_sheet_existing_primary = []
new_sheet_new_primary = []
for a_row in test_grant_primary.values():
    #check excel
    existing_item = [i for i in google_sheet if a_row['title'] == i['Title']]
    if existing_item:
        new_sheet_existing_primary.append(existing_item)
    else:
        new_sheet_new_primary.append(a_row)
        print('No item')

In [ ]:
# Comparing secondary awards with existing google sheets

In [ ]:
test_grant_secondary = pubs_when_grant_secondary['DK107980'] 

In [ ]:
print(len(test_grant_secondary))

In [ ]:
new_sheet_existing_secondary = []
new_sheet_new_secondary = []
for a_row in test_grant_secondary.values():
    #check excel
    print(a_row)
    existing_item = [i for i in google_sheet if a_row['title'] == i['Title']]
    if existing_item:
        new_sheet_existing_secondary.append(existing_item)
    else:
        new_sheet_new_secondary.append(a_row)
        print('No item')

In [ ]:
# Adding the publications in the google sheets that are not in the primary_awards

In [ ]:
gs_only_items = []
for a_row in google_sheet:
    if a_row['Title'] == '':
        continue
    existing_item_1 = [i for i in test_grant_primary.values() if a_row['Title'] == i['title']]
    existing_item_2 = [i for i in test_grant_secondary.values() if a_row['Title'] == i['title']]
    if existing_item_1 or existing_item_2:
        continue
    else:
        gs_only_items.append(a_row)
        

In [ ]:
print(gs_only_items)

In [ ]:
pubs_when_grant_secondary[]

In [ ]:
for ele in test_grant_secondary.values():
    print(ele['title'], ele['primary_awards'])

In [ ]:
print(len(pubs_when_grant_secondary.keys()))

In [ ]:
print(len(pubs_when_grant_primary.keys()))

In [ ]:
# for key in pubs_when_grant_secondary:
#     print(key,len(pubs_when_grant_secondary[key].keys()))

In [ ]:
keys_primary = set(pubs_when_grant_primary.keys())

In [ ]:
print(keys_primary)

In [ ]:
keys_secondary = set(pubs_when_grant_secondary.keys())

In [ ]:
print(keys_primary.difference(keys_secondary))

In [ ]:
# Write to the google sheet

In [ ]:
new_worksheet = test_spreadsheet.add_worksheet('collate_v2_test', 100, 15)

In [ ]:
col_order = {'title': 1, 'authors': 2, 'date':3, 'contact_4DN_author':4,'primary_awards':len(col_names) - 4, 'secondary_awards':len(col_names) - 3, 'other_ids':len(col_names) - 2, 'link':len(col_names) - 1, 'date_added': len(col_names)}
gs_write = []
row = 1
# Writting the existing info 
for r,line in enumerate(new_sheet_existing_primary):
    row = r + 1
    if row == 1: # Writes the labels of the columns (the keys)
        for c, key in enumerate(line[0]):
            col = c + 1
            gs_write.append(gspread.models.Cell(row, col, key))
    row = r + 2
    for c, key in enumerate(line[0]):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[0][key]))     
# Writing existing info in gs only
for line in gs_only_items:
    row = row + 1
    for c, key in enumerate(line):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[key]))
        
# Writting new info
for line in new_sheet_new_primary:
    row = row + 1
    for c, key in enumerate(line):
        #col = c + 1
        col = col_order[key]
        gs_write.append(gspread.models.Cell(row, col, line[key]))
    
#Writting message
row = row + 1
spaces = ['' for x in range(len(col_names))]
message = ['Publications in which this award is listed but it is not primary' if x == 0 else '' for x in range(len(col_names))] 

for c, txt in enumerate(spaces):
    col = c + 1
    gs_write.append(gspread.models.Cell(row,col,txt))
    
row = row + 1   
for c, txt in enumerate(message):
    col = c + 1
    gs_write.append(gspread.models.Cell(row,col,txt))

# Writting secondary awards existing info 
for r, line in enumerate(new_sheet_existing_secondary):
    row = row + 1
    for c, key in enumerate(line[0]):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[0][key]))
        
# Writting secondary awards new info 
for r, line in enumerate(new_sheet_new_secondary):
    row = row + 1
    for c, key in enumerate(line):
#         col = c + 1
        col = col_order[key]
        gs_write.append(gspread.models.Cell(row, col, line[key]))

        
#worksheet.update_cells(gs_write,value_input_option='USER_ENTERED')

In [ ]:
print(spaces)

In [ ]:
print(message)

In [ ]:
spaces = [''] * 10

In [ ]:
print(new_sheet_new_primary)

In [ ]:
column_df = [''] * len(df['Title']) 
    for item in links_col:
        column_df[item[0]] = item[1]
    df['Portal link'] = column_df
    
        # Arrange columns in the right order
    col_order_first = ['Title', 'Authors', 'Date', 'Contact 4DN Author']
    col_order_middle = []
    col_order_last = ['Primary_awards', 'Secondary Awards', 'Other ids','Link','Date added']

In [ ]:
print(col_names)

In [ ]:
col_order = {}
# for i,col in enumerate(col_names):
#     col_order[col] = 

In [ ]:
print(col_order)

In [ ]:
col_order = {col:i for i,col in enumerate(col_names, 1)}

In [ ]:
print(col_order)

In [ ]:
col_order2 = {col:i for i,col in enumerate(test_grant_primary.keys, 1)}

In [ ]:
len(col_names)

In [ ]:
col_order = {}
for x in range(len(col_names), 1):
    
    
    

In [ ]:
test_grant_primary.keys()

In [ ]:
col_order = {'title': 1, 'authors': 2, 'date':3, 'contact_4DN_author':4,'primary_awards':len(col_names) - 4, 'secondary_awards':len(col_names) - 3, 'other_ids':len(col_names) - 2, 'link':len(col_names) - 1, 'date_added': len(col_names)}

In [ ]:
################### HERE ###################################

In [1]:
import collections
import os
import json
from difflib import SequenceMatcher as SM
from itertools import combinations
from datetime import date
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time
import pandas as pd
from collections import OrderedDict
from gspread_formatting import *
import string

In [2]:
pubs_when_grant_primary = collections.defaultdict(dict)
pubs_when_grant_secondary_tracker = collections.defaultdict(dict)
pubs_when_grant_secondary = collections.defaultdict(dict)

fname_grant_jsons = os.listdir("/Users/luisa/pub_tracker_backup/collate_revisited_test/data_grant")
for fname_json in fname_grant_jsons:
    with open("/Users/luisa/pub_tracker_backup/collate_revisited_test/data_grant/" + fname_json, "r") as fp:
        dic_pergrant = json.load(fp)
        pubs = dic_pergrant.keys()
        grant_name = fname_json[:-5]
        pubs_when_grant_primary[grant_name] = collections.defaultdict(dict)
        pubs_when_grant_secondary[grant_name] = collections.defaultdict(dict)
        for pub in pubs:
            authors = "; ".join(dic_pergrant[pub]["authors"])
            title = dic_pergrant[pub]["title"]
            link = dic_pergrant[pub]["link"]
            date = dic_pergrant[pub]["date"]
            other_ids = ";".join(dic_pergrant[pub]["other_ids"])
            primary_awards = ";".join(dic_pergrant[pub]["primary_awards"])
            secondary_awards = ";".join(dic_pergrant[pub]["secondary_awards"])
            contact_4DN_author = dic_pergrant[pub]["4DN_member_author"]
            added_date = dic_pergrant[pub]["added"]

            pubs_when_grant_primary[grant_name][pub]['title'] = title
            pubs_when_grant_primary[grant_name][pub]['authors'] = authors
            pubs_when_grant_primary[grant_name][pub]['date'] = date
            pubs_when_grant_primary[grant_name][pub]['link'] = link
            pubs_when_grant_primary[grant_name][pub]['other_ids'] = other_ids
            pubs_when_grant_primary[grant_name][pub]['primary_awards'] = primary_awards
            pubs_when_grant_primary[grant_name][pub]['secondary_awards'] = secondary_awards
            pubs_when_grant_primary[grant_name][pub]['date_added'] = added_date
            pubs_when_grant_primary[grant_name][pub]['contact_4DN_author'] = contact_4DN_author
            
#     # Keeps track of the grants of the publications secondary awards
    
for grant in pubs_when_grant_primary:
    for pub in pubs_when_grant_primary[grant]:
        if pubs_when_grant_primary[grant][pub]['secondary_awards'] != '':
            if ';' not in pubs_when_grant_primary[grant][pub]['secondary_awards']:
                sec_award = pubs_when_grant_primary[grant][pub]['secondary_awards']
                pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub]
            else:
                sec_award_list = pubs_when_grant_primary[grant][pub]['secondary_awards'].split(';')
                for sec_award in sec_award_list:
                    pubs_when_grant_secondary[sec_award][pub] = pubs_when_grant_primary[grant][pub]

In [4]:
print(len(pubs_when_grant_primary['DK107980']))

54


In [5]:
print(len(pubs_when_grant_secondary['DK107980']))

14


In [6]:
# Accesses to the google sheets
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)
f = open('/Users/luisa/pub_tracker_final_test/google_sheet_link.txt', 'r')
link = f.readline().split('\n')
google_management_sheet_id = link[0]
f.close()
google_management_spreadsheet = gc.open_by_key(google_management_sheet_id)
google_management_sheet = google_management_spreadsheet.sheet1
grant_sheets_links = google_management_sheet.get_all_values()

In [19]:
reqst_ct = 0
log = []
for grant in pubs_when_grant_primary:
    if grant != 'DK107980':
        continue
    for item in grant_sheets_links:
        if grant in item[1]:
            spreadsheet_id = item[2]
            if reqst_ct >= 60:
                time.sleep(100)
                reqst_ct = 0
                spreadsheet = gc.open_by_key(spreadsheet_id)
                print('Working on ', spreadsheet)
                reqst_ct += 1
            else:
                spreadsheet = gc.open_by_key(spreadsheet_id)
                print('Working on ', spreadsheet)
                reqst_ct += 1

            if reqst_ct >= 60:
                time.sleep(100)
                reqst_ct = 0
                try:
                    worksheet = spreadsheet.worksheet('Publication List')
                    reqst_ct += 1
                except:
                    worksheet = spreadsheet.add_worksheet(title='Publication List', rows='200', cols='15')
                    reqst_ct += 1
            else:
                try:
                    worksheet = spreadsheet.worksheet('Publication List')
                    reqst_ct += 1

                except:
                    worksheet = spreadsheet.add_worksheet(title='Publication List', rows='200', cols='15')
                    reqst_ct += 1
                    
            result = update_google_sheets()
            print(result + 'in ' + item[0])
            print(grant)
            print()

Working on  <Spreadsheet 'NOFIC - Dekker Center (1U54DK107980-01)' id:1v4J-2a5e6a3mbZAI2B3Yk2z1WFdQI5ZNktlXCbx_MOM>
0 new primary publications
0 new secondary publications
0 new primary publications and 0 new secondary publicationsin NOFIC - Dekker Center (1U54DK107980-01)
DK107980



In [18]:
def update_google_sheets():
    log = []
    # Read the existing google sheet
    #worksheet = test_spreadsheet.worksheet("Publication_List_new_format_updated")
    worksheet_values = worksheet.get_all_values()
    df = pd.DataFrame(worksheet_values[1:], columns=worksheet_values[0])
    col_names = df.columns.values
    google_sheet = df.to_dict(orient='records', into=OrderedDict)
    
    # Comparing primary awards with existing google sheets and adding new awards
    new_sheet_existing_primary = []
    new_sheet_new_primary = []
    for a_row in pubs_when_grant_primary[grant].values():
        #check excel
        existing_item = [i for i in google_sheet if a_row['title'] == i['Title']]
        if existing_item:
            new_sheet_existing_primary.append(existing_item)
        else:
            new_sheet_new_primary.append(a_row)
    print('{} new primary publications'.format(len(new_sheet_new_primary)))

    # Comparing secondary awards with existing google sheets and adding new awards
    new_sheet_existing_secondary = []
    new_sheet_new_secondary = []
    for a_row in pubs_when_grant_secondary[grant].values():
        #check excel
        existing_item = [i for i in google_sheet if a_row['title'] == i['Title']]
        if existing_item:
            new_sheet_existing_secondary.append(existing_item)
        else:
            new_sheet_new_secondary.append(a_row)
    print('{} new secondary publications'.format(len(new_sheet_new_secondary)))
    
    # Adding the existing publications in the google sheets that are not in primary awards (added manually)
    gs_only_items = []
    for a_row in google_sheet:
        if a_row['Title'] == '' or a_row['Title'] == 'Publications in which this award is listed but it is not primary':
            continue
        existing_item_1 = [i for i in pubs_when_grant_primary[grant].values() if a_row['Title'] == i['title']]
        existing_item_2 = [i for i in pubs_when_grant_secondary[grant].values() if a_row['Title'] == i['title']]
        if existing_item_1 or existing_item_2:
            continue
        else:
            gs_only_items.append(a_row)
    
    #Writting info to the google sheets
    col_order = {'title': 1, 'authors': 2, 'date':3, 'contact_4DN_author':4,'primary_awards':len(col_names) - 4, 'secondary_awards':len(col_names) - 3, 'other_ids':len(col_names) - 2, 'link':len(col_names) - 1, 'date_added': len(col_names)}
    gs_write = []
    row = 1
    fmt_bold = cellFormat(textFormat=textFormat(bold=True))
    # Writting the existing info 
    for r,line in enumerate(new_sheet_existing_primary):
        row = r + 1
        if row == 1: # Writes the labels of the columns (the keys)
            for c, key in enumerate(line[0]):
                col = c + 1
                gs_write.append(gspread.models.Cell(row, col, key))
        row = r + 2
        for c, key in enumerate(line[0]):
            col = c + 1
            gs_write.append(gspread.models.Cell(row, col, line[0][key]))     
    # Writing existing info in gs only
    for line in gs_only_items:
        row = row + 1
        for c, key in enumerate(line):
            col = c + 1
            gs_write.append(gspread.models.Cell(row, col, line[key]))

    # Writting new info
    for line in new_sheet_new_primary:
        row = row + 1
        for key in line:
            col = col_order[key]
            gs_write.append(gspread.models.Cell(row, col, line[key]))

    #Writting message
    row = row + 1
    spaces = ['' for x in range(len(col_names))]
    message = ['Publications in which this award is listed but it is not primary' if x == 0 else '' for x in range(len(col_names))] 

    for c, txt in enumerate(spaces):
        col = c + 1
        gs_write.append(gspread.models.Cell(row,col,txt))
        

    row = row + 1   
    for c, txt in enumerate(message):
        col = c + 1
        gs_write.append(gspread.models.Cell(row,col,txt))
        format_range_message = [('A{}'.format(row),fmt_bold)]

    # Writting secondary awards existing info 
    for r, line in enumerate(new_sheet_existing_secondary):
        row = row + 1
        for c, key in enumerate(line[0]):
            col = c + 1
            gs_write.append(gspread.models.Cell(row, col, line[0][key]))

    # Writting secondary awards new info 
    for r, line in enumerate(new_sheet_new_secondary):
        row = row + 1
        for c, key in enumerate(line):
    #         col = c + 1
            col = col_order[key]
            gs_write.append(gspread.models.Cell(row, col, line[key]))


    worksheet.update_cells(gs_write,value_input_option='USER_ENTERED')
#Adding formatting to worksheet
    num_to_letter_dict = {x:y for x, y in zip(range(1, 27), string.ascii_uppercase)}
    num_rows = len(gs_write)//len(df.columns)
    fmt_plain = cellFormat(textFormat=textFormat(bold=False))
    
    range_worksheet_formatting = [('A1:{}{}'.format(num_to_letter_dict[len(df.columns)],num_rows),fmt_plain)]
    range_title_formatting = [('A1:{}1'.format(num_to_letter_dict[len(df.columns)]), fmt_bold)]
    format_cell_ranges(worksheet,range_worksheet_formatting) #Clean the formatting of worksheet
    format_cell_ranges(worksheet,format_range_message) # Make the title bold
    format_cell_ranges(worksheet,range_title_formatting) # Make the subtitle bold
    return '{} new primary publications and {} new secondary publications'.format(len(new_sheet_new_primary), len(new_sheet_new_secondary))

#     new_pubs_added_primary = []
#     new_pubs_added_secondary = []
#     islog = False
#     if len(new_sheet_new_primary) >= 1:
#         islog = True
#         new_pubs_added_primary = [pub for pub in new_sheet_new_primary]
#     if len(new_sheet_new_secondary) >= 1:
#         islog = True
#         new_pubs_added_secondary = [pub for pub in new_sheet_new_secondary]
#     log.append(new_pubs_added_primary + new_pubs_added_secondary)
#     print('{} new primary publications and {} new secondary publications'.format(len(new_sheet_new_primary), len(new_sheet_new_secondary)))
#     if islog:
#         return log
    

In [ ]:
results = update_google_sheets()

In [ ]:
print(result)

In [ ]:
print(log)

In [ ]:
#Writting info to the google sheets
col_order = {'title': 1, 'authors': 2, 'date':3, 'contact_4DN_author':4,'primary_awards':len(col_names) - 4, 'secondary_awards':len(col_names) - 3, 'other_ids':len(col_names) - 2, 'link':len(col_names) - 1, 'date_added': len(col_names)}
gs_write = []
row = 1
fmt_bold = cellFormat(textFormat=textFormat(bold=True))
# Writting the existing info 
for r,line in enumerate(new_sheet_existing_primary):
    row = r + 1
    if row == 1: # Writes the labels of the columns (the keys)
        for c, key in enumerate(line[0]):
            col = c + 1
            gs_write.append(gspread.models.Cell(row, col, key))
    row = r + 2
    for c, key in enumerate(line[0]):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[0][key]))     
# Writing existing info in gs only
for line in gs_only_items:
    row = row + 1
    for c, key in enumerate(line):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[key]))

# Writting new info
for line in new_sheet_new_primary:
    row = row + 1
    for key in line:
        col = col_order[key]
        gs_write.append(gspread.models.Cell(row, col, line[key]))

#Writting message
row = row + 1
spaces = ['' for x in range(len(col_names))]
message = ['Publications in which this award is listed but it is not primary' if x == 0 else '' for x in range(len(col_names))] 

for c, txt in enumerate(spaces):
    col = c + 1
    gs_write.append(gspread.models.Cell(row,col,txt))


row = row + 1   
for c, txt in enumerate(message):
    col = c + 1
    gs_write.append(gspread.models.Cell(row,col,txt))
    format_range_message = [('A{}'.format(row),fmt_bold)]

# Writting secondary awards existing info 
for r, line in enumerate(new_sheet_existing_secondary):
    row = row + 1
    for c, key in enumerate(line[0]):
        col = c + 1
        gs_write.append(gspread.models.Cell(row, col, line[0][key]))

# Writting secondary awards new info 
for r, line in enumerate(new_sheet_new_secondary):
    row = row + 1
    for c, key in enumerate(line):
#         col = c + 1
        col = col_order[key]
        gs_write.append(gspread.models.Cell(row, col, line[key]))
        
new_worksheet.update_cells(gs_write,value_input_option='USER_ENTERED')

#Adding formatting to worksheet
num_to_letter_dict = {x:y for x, y in zip(range(1, 27), string.ascii_uppercase)}
num_rows = len(gs_write)//len(df.columns)
fmt_plain = cellFormat(textFormat=textFormat(bold=False))

range_worksheet_formatting = [('A1:{}{}'.format(num_to_letter_dict[len(df.columns)],num_rows),fmt_plain)]
range_title_formatting = [('A1:{}1'.format(num_to_letter_dict[len(df.columns)]), fmt_bold)]

format_cell_ranges(worksheet,range_worksheet_formatting) #Clean the formatting of worksheet
format_cell_ranges(worksheet,format_range_message) # Make the title bold
format_cell_ranges(worksheet,range_title_formatting) # Make the subtitle bold
